## Week06, More Data Preparation (regression model)

ISM6136

&copy; 2023 Dr. Tim Smith

---

## More on data preparation

Last week, we looked at finding and droping missing values before splitting the data.

This week, we will look at some other data preparation steps. This also includes making sure that ANY DATA TRANSFORMATION or VALUE IMPUTATION is calculated after the train-test split, and IS APPLIED TO BOTH TRAINING AND TEST DATA. 

### Import Modules

In [89]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

from matplotlib import pyplot as plt

np.random.seed(42)


### Load Data

In [90]:
df = pd.read_csv('./data/data-regression.csv', index_col=False)

In [91]:
df.head(10)

,feature1,feature2,feature3,feature4,target
0,355.19,blue,NaN,0.0,-143.065368
1,308.12,green,-16.278,8.0,-4.461594
2,500.51,green,NaN,8.0,-0.835365
3,451.46,green,-8.159,NaN,-34.189506
4,502.45,green,18.650,NaN,26.118365
5,597.51,green,13.858,6.0,NaN
6,238.03,NaN,-98.232,5.0,97.595136
7,488.55,Red,-87.694,-12.0,72.342689
8,439.83,red,NaN,-1.0,123.887711
9,591.54,green,5.088,NaN,-13.667815


In [92]:
df.shape

(100, 5)

## Address any NaN values that need to be dropped from the dataset

Count the number of missing values for each feature.

In [93]:
df.isna().sum()

feature1    10
feature2    10
feature3    10
feature4    10
target      10
dtype: int64

Observations with missing Target values should be dropped from the dataset.

In [94]:
df = df.dropna(subset=['target'])

Re-look at the data. Notice that there are some features that have a lot of missing values.

In [95]:
df.isna().sum()

feature1     8
feature2    10
feature3     9
feature4     8
target       0
dtype: int64

If the number of missing values in a feature is high (relative to the number of observations), then remove that feature from the dataset. Here I've used 60%, but what you use will depend on the dataset and the context of the problem/analysis.

In [96]:
df = df.dropna(axis=1, thresh=int(0.60*df.shape[0])) #axis=1 is column. thresh=int(0.60*df.shape[0]) means that if a column has less than 60% of the data, it will be dropped

If the number of missing values in a observation (row) is high

In [97]:
df = df.dropna(axis=0, thresh=int(0.25*(df.shape[1]-1))) # axis=0 is row. thresh=int(0.25*(df.shape[1]-1)) means that if a row has less than 25% of the data, it will be dropped

In [98]:
df.shape # check how many columns and rows are left

(90, 5)

In [99]:
df.isna().sum() # check how many missing values are left in each column

feature1     8
feature2    10
feature3     9
feature4     8
target       0
dtype: int64

NOTE: The remaining missing values will be imputed. **This is done after the train-test split**

## Address any Misspelled words

The Unique method will give us the unique values in a column. This will help us identify any misspelled words. We can then use the replace method to replace any misspelled words.

In [100]:
df['feature2'].unique()

array(['blue', 'green', nan, 'Red', 'red'], dtype=object)

In [101]:
df = df.replace(['Red'], 'red') 

### Turn categorical variables into dummy variables

Here we will use the get_dummies method to turn categorical variables into dummy variables.

First, we will encode the feature2 catagorical variable.

In [102]:
df = pd.get_dummies(
    df, 
    prefix_sep='_', 
    dummy_na=False, 
    drop_first=True, 
    columns=['feature2'], 
    dtype='int32'
)

df.head()

,feature1,feature3,feature4,target,feature2_green,feature2_red
0,355.19,NaN,0.0,-143.065368,0,0
1,308.12,-16.278,8.0,-4.461594,1,0
2,500.51,NaN,8.0,-0.835365,1,0
3,451.46,-8.159,NaN,-34.189506,1,0
4,502.45,18.650,NaN,26.118365,1,0


## Conduct your Train-Test Split BEFORE standardizing the data or imputing mnissing values

The Train-Test Split should be conducted before any data cleaning that is based on calculations of the data. For instance, imputing missing values, and standardizing/normalizing data.


In [103]:
features = df.drop(columns=['target'])
target = df['target']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [104]:
X_train.columns

Index(['feature1', 'feature3', 'feature4', 'feature2_green', 'feature2_red'], dtype='object')

### Impute any missing values

Next, we imput missing values. For numeric columns, we will use the mean value. For the dummy variable columns, we will use 0 for missing values.

In [105]:
# for the dummy variables, we choose zero for our impute value. 
for column in X_train.select_dtypes(include=['object']):
    train_mode = X_train[column].value_counts().index[0]
    X_train[column] = X_train[column].fillna(0)
    X_test[column] = X_test[column].fillna(0)

# for the numeric columns, we choose the mean for our impute value.
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

numeric_cols = ['feature1', 'feature3', 'feature4']
imp.fit(X_train[numeric_cols])
X_train[numeric_cols] = imp.transform(X_train[numeric_cols])
X_test[numeric_cols] = imp.transform(X_test[numeric_cols])

X_test.head()

,feature1,feature3,feature4,feature2_green,feature2_red
47,581.29,-35.365000,14.0,0,1
25,509.18,-5.050000,6.0,0,0
63,614.28,38.434000,5.0,0,1
80,627.77,-25.866000,5.0,0,0
0,355.19,-26.060754,0.0,0,0


### Scale the data

For our final step in data preparation, we scale the data. We will use the StandardScaler from sklearn.preprocessing to scale the data.

In [106]:
# scale the numeric features

scaler = StandardScaler()
scaler.fit(X_train[numeric_cols])
X_train[numeric_cols] = scaler.transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

X_train.head()

,feature1,feature3,feature4,feature2_green,feature2_red
56,2.085521e+00,-9.468528e-01,1.945161,0,0
71,1.159977e-15,7.698459e-01,-0.888008,0,1
83,5.494188e-01,2.931571e-01,0.713348,0,1
79,-1.871408e+00,-8.027772e-17,0.343804,0,0
86,3.098461e-01,-8.027772e-17,-0.888008,0,0


### Fit the model

Finally, we fit the model. We will use the LogisticRegression model from sklearn.linear_model.

In [107]:
model = LinearRegression(
    n_jobs=-1       # use all processors
)
_ = model.fit(X_train, y_train)

In [108]:
results = pd.DataFrame()
results['actual'] = y_test
results['predicted'] = model.predict(X_test)
results.head()

,actual,predicted
47,104.746909,145.450337
25,-175.916416,-74.168783
63,38.071656,41.356899
80,-41.610828,-59.747269
0,-143.065368,-78.164125


In [111]:
# calculate the root mean square error

from sklearn.metrics import mean_squared_error

print(f"The root mean squared error for this model is {mean_squared_error(results['actual'], results['predicted'], squared=False):0.3f}")


The root mean squared error for this model is 53.248
